### PyTorch によるモデル作成の手順
#### 1. 必要なものを定義
- Dataset, transform, DataLoader
- Model
- Loss
- Optimizer
- Scheduler
- device

#### 2. 訓練フローの構築
- 訓練フロー
- 検証フロー

#### 3. 実行
- モデルの保存
- 実験管理

---
#### 1-1-1. Dataset の定義
この3パターンのどれかで取得する
1. 公式からimport
2. ImageFolderを使う
3. 自作

1. 公式からimport

    PyTorch の torchvisionには、代表的な画像データセット（MNIST、CIFAR-10、ImageNet など）が用意されており、簡単に利用できる．
    - 8-9割はこれ．
    - root は dataset が属するディレクトリ. dataset 自体のパスじゃない
    - download=True で，パスになかったら DL してくれる

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import mlflow
import mlflow.sklearn

In [ ]:
import torchvision

train_ds = torchvision.datasets.CIFAR10(root = "path/to/datasets", train=True, download=True)
val_ds = torchvision.datasets.CIFAR10(root = "path/to/datasets", train=False)

2. ImageFolder を使う

    こんな構造のディレクトリのパスを渡すと，    
    自動でクラスを割り当て，画像を読み取ってくれる．

    <img src="imagefolder.png" width="300">

In [ ]:
from torchvision.datasets import ImageFolder

train_ds = ImageFolder(root="path/to/animal_dataset")

3. 自作

    1, 2 が使えないならオリジナルの Datasetクラス を自作するのが基本．

    必須要件

    - `__len__(self)`    
        データセットの総サンプル数を返す関数が必要．len(dataset) が機能するように

    - `__getitem__(self, idx)`
    
        指定したインデックスのサンプルを返す必要がある．        
        dataset[0] や dataset[200] のようにデータにアクセスしたり，`for`文でイテレートできるように

    推奨事項
    - `transform` や `target_transform` を `__init__` で受け取り,`__getitem__` 内でデータに適用させること
    - 画像データセットとして定義する場合，`__getitem__` は，画像（`PIL.Image.Image`）と正解ラベル（`int`）の **タプル** を返すこと．



In [ ]:
# カスタムデータセットの例
from torch.utils.data import Dataset

class SimpleDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')  # RGBに変換
        if self.transform:
            image = self.transform(image)
        return image, label

---
#### 1-1-2. transform の定義
- データに対して行いたい前処理がある場合，`Dataset`の`transform`や`target_transform`に前処理を定義することができる．

- `torchvision.transforms` を用い、`Compose` で複数の操作をまとめて定義する．

- 画像データに対しては，`PIL.Image.Image`型のデータを，平均0, 分散1の`Tensor`に `Normalize` したうえで，必要に応じてデータ拡張を行う．

- `Normalize` の引数は使うデータセットによって変わる．調べて適切な値を入れる必要がある．

- 検証用データセットにデータ拡張かけないように
    - `train_ds`: `Data Augmentation` + `ToTensor` + `Normalize`
    - `val_ds`: `ToTensor` + `Normalize`

In [ ]:
# transform の例
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),                           # 入力サイズを統一
    transforms.RandomHorizontalFlip(p=0.5),                  # 水平反転によるデータ拡張
    transforms.ColorJitter(brightness=0.2, contrast=0.2),   # 明るさ・コントラストをランダムに変化
    transforms.ToTensor(),                                   # PIL→Tensor（値を [0,1] にスケーリング）
    transforms.Normalize(mean=[0.485, 0.456, 0.406],         # ImageNet 標準の平均
                         std=[0.229, 0.224, 0.225])         # ImageNet 標準の分散
])

# データセットに適用
train_ds = torchvision.datasets.CIFAR10(root = "path/to/datasets", train=True, transform=transform)

---
#### 1-1-3. DataLoader の定義
- 作った`Dataset`を`DataLoader`の引数に渡すことで，ミニバッチを作ったり，ミニバッチの内容をシャッフルするのに必要．

- 引数を変えるだけで高速化．とりあえず書いとけってのがある．



In [ ]:
# DataLoader の例
from torch.utils.data import DataLoader

batch_size = 128

train_dl = DataLoader(
    train_ds,               # Dataset オブジェクト
    batch_size=batch_size,         # ミニバッチのサイズ
    shuffle=True,          # エポックごとにシャッフル（デフォルトでTrue）
    num_workers=2,         # 高速化：並列データ読み込み数 2が一番早いとか？
    pin_memory=True,       # 高速化：GPU 転送を最適化
)

---
#### 1-1-a. データセット定義の完成例

訓練用データセット `train_ds`, 検証用データセット `val_ds`

それぞれに対応する`DataLoader`である，`train_dl`, `val_dl` を定義

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4), 
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5070, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2761]) # CIFAR100用
    ])
val_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5070, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2761]) # CIFAR100用
    ])

train_ds = torchvision.datasets.CIFAR10(root="path/to/datasets", train=True, transform=train_tf, download=True)
val_ds = torchvision.datasets.CIFAR10(root="path/to/datasets", train=False, transform=val_tf)

batch_size = 128
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)


---
#### 1-2. Model の定義

パターンは様々だが，だいたい次の3つ
1. 公式からimport
2. 外部からもってくる
3. 自作

1. 公式からimport

    - PyTorch の torchvision.models には、代表的なモデル（ResNet、VGG、AlexNet、MobileNet など） が用意されており、簡単に利用できる。

    - pretrained=True で、ImageNetで事前学習された重みをロードできる。

    - クラス数を指定する引数を必ずいれること！！入れないとImageNet用の1000クラス分類されちゃう．
        - だいたい `num_classes` で指定できる．
        - モデルの最後の全結合層の出力チャネル数を変更してる -> クラス数と同じ数の出力チャネルとなる

In [ ]:
# import の例
from torchvision import models
model = models.resnet18(num_classes=10)  # 10クラス分類用

2. 外部からもってくる

    - GitHubリポジトリなどからも実装を持ってこれる

    - PyTorchに実装されていない新しめのモデルやカスタム実装が必要な時によく使う（例：CIFAR系のような，小さい画像を分類するためのアーキテクチャ）

    - だいたい `models/` に `xxx.py` がいっぱい入ってる．そこをあさればお目当てのが見つかりやすいかも

    - 環境に`.py`配置して `import` するとか，ターミナルからコマンドたたくとか，いろいろ実行方法あり

    Ex.) CIFAR向けのアーキテクチャ：https://github.com/weiaicunzai/pytorch-cifar100

3. 自作

    - `nn.Module`を継承する．

    - `foward` は必ずオーバーライド．バッチ次元が含まれたテンソルに対して処理を書く必要がある．

    - `__init__` に使う層を書いて，`foward` に処理の過程を書くのがセオリー
    
    詳細はPyTorch Tutorials に任せる．すこし調べて簡単なCNNが書けるくらいなら十分な気がする．

    https://docs.pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html


---
#### 1-3. Loss の定義
この2つ以外はほぼ使われないとおもわれる．

- 分類タスク (離散値を予測) -> CrossEntoropyLoss

- 回帰タスク (連続値を予測) -> MSELoss (平均二乗誤差)

    注意：CrossEntoropyLossはモデルの出力(logit)を直接受け取ることを前提としている．\
    SoftmaxをとおしてからCrossEntoropyLossをとおすと正しい損失が算出できなくなる．\
    すでにCrossEntoropyLossは内部にSoftmaxの処理がある．

In [ ]:
import torch
criterion = torch.nn.CrossEntropyLoss()

---
#### 1-4. Optimizer の定義
第一引数に更新対象のパラメータのジェネレータを指定する．例示した通りに指定すればok

さまざまな Optimizer があるが，研究でよく使われるのは次のとおり：

CNN系：momentum SGD, Adam, AdamW\
Transformer系：AdamW

- SGD (w/ momentum/Nesterov)\
汎化性能・安定性・計算コストの面で最強．だが，収束がかなり遅い

- Adam\
非常に速く収束する．だが，汎化性能は微妙．学習安定性も高くない

- AdamW\
Adamに重み減衰加えたやつ．Adamよりも汎化性能・安定性が高い．かなり人気らしい．transformer系は特に



In [ ]:
import torch
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True) # よく使われる設定

---
#### 1-5. Scheduler の定義
Optimizer の学習率を上書きして変化させる．使わなくても訓練できるため使用必須ではない．\
だが，使うとだいたい精度上がるし，研究でも採用されることが多い．

とりあえずCosineAnneling (+ WarmUp ?) を使えば問題ない．\
https://docs.pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html\
https://docs.pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingWarmRestarts.html#torch.optim.lr_scheduler.CosineAnnealingWarmRestarts

SGDとは相性バッチリで，Adam, AdamWとも合わんわけではないらしい．

注意：基本的に epoch ごとに学習率を変化させるが，たまに iteration ごとに変化させるのを前提としたものがある．この場合，scheduler.step()を呼ぶ位置を変える必要がある．Ex.) OneCycleLR

In [ ]:
import torch
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0, last_epoch=-1)

---
#### 1-6. device の定義
PyTorchでは，GPUを活用してモデルの訓練を行うことが可能である．ただし，演算に使用するすべてのテンソルは，同じデバイス（CPUやGPUなど）上に配置されている必要がある．

そのため，あらかじめ使用するデバイスを変数として定義しておき，テンソルの配置先を統一しておくと便利である．

なお，`.to(device)` でテンソルのデバイスを合わせなければいけないのは，"入力データ" と "モデル" である．

以下に一般的な記述例を示す．

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU があれば使う

---
#### 1-a. 定義の完成例
モデル作成に必要なパーツを定義する例を示す．

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

epochs = 100
num_classes = 10
lr = 0.005
batch_size = 128

train_tf = transforms.Compose([]) # 省略
val_tf = transforms.Compose([]) # 省略

train_ds = torchvision.datasets.CIFAR10(root="path/to/datasets", train=True, transform=train_tf, download=True)
val_ds = torchvision.datasets.CIFAR10(root="path/to/datasets", train=False, transform=val_tf)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

model = torchvision.models.resnet18(num_classes=num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0, last_epoch=-1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

---
#### 2-1. 訓練フローの定義


In [ ]:
# 訓練フローを定義
def train_1epoch(model, train_dl, criterion, optimizer, scheduler=None, device="cpu"):
    total_loss = 0.0
    total_corr = 0

    model.train()                                       # 訓練モードに設定
    for inputs, labels in train_dl:                     # ミニバッチを取得してループ
        inputs = inputs.to(device)                      # GPU に転送
        labels = labels.to(device)                      # GPU に転送

        outputs = model(inputs)                         # 順伝播

        loss = criterion(outputs, labels)               # 損失計算
        preds = torch.argmax(outputs.detach(), dim=1)   # 予測値を取得
        corr = torch.sum(preds == labels.data).item()   # 正解数をカウント

        optimizer.zero_grad()                           # 勾配を初期化
        loss.backward()                                 # 誤差逆伝播
        optimizer.step()                                # 重み更新

        total_loss += loss.item() * len(inputs)         # 損失を累積
        total_corr += corr                              # 正解数を累積

    if scheduler is not None:                           # 学習率スケジューラが指定されている場合
        scheduler.step()                                # 学習率を更新
    train_loss = total_loss / len(train_dl.dataset)     # 平均損失を計算
    train_acc = total_corr / len(train_dl.dataset)      # 平均精度を計算

    return train_loss, train_acc

---
#### 2-2. 検証フローの定義


In [ ]:
# 検証フローを定義
def val_1epoch(model, val_dl, criterion, device="cpu"):
    total_loss = 0.0
    total_corr = 0

    model.eval()                                            # 評価モードに設定
    with torch.no_grad():                                   # 勾配計算を無効化
        for inputs, labels in val_dl:                       # ミニバッチを取得してループ
            inputs = inputs.to(device)                      # GPU に転送
            labels = labels.to(device)                      # GPU に転送
            
            outputs = model(inputs)                         # 順伝播

            loss = criterion(outputs, labels)               # 損失計算
            preds = torch.argmax(outputs.detach(), dim=1)   # 予測値を取得
            corr = torch.sum(preds == labels.data).item()   # 正解数をカウント

            total_loss += loss.item() * len(inputs)         # 損失を累積
            total_corr += corr                              # 正解数を累積

    val_loss = total_loss / len(val_dl.dataset)             # 平均損失を計算
    val_acc = total_corr / len(val_dl.dataset)              # 平均精度を計算

    return val_loss, val_acc

---
#### 2-3. 訓練


In [ ]:
# 訓練
model = model.to(device)
for epoch in range(epochs):
    train_loss, train_acc = train_1epoch(model, train_dl, criterion, optimizer, scheduler, device)
    val_loss, val_acc = val_1epoch(model, val_dl, criterion, device)
    print(f"epoch {epoch+1:>3}/{epochs:>3}: train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")


---
#### 3-1. モデルの保存
一般的に，`model`の`state_dict`と呼ばれるものを保存するのが一般的である．

state_dict を保存しておけば，モデルのパラメータやバッファが保存され，同じ状態を後で再現できるようになる．

モデルをロードする際，形状が合わないとエラーが発生して読み込めない．

In [ ]:
import torch

torch.save(model.state_dict(), "tmp.pth") # モデルの重みを保存
model.load_state_dict(torch.load("tmp.pth")) # モデルの重みを読み込み

---
#### 3-2. 実験管理
研究を始めると，様々な条件やハイパーパラメータを調整して実験する．\
実験結果や，学習環境を覚えておくことは無理．\
そこで，うまく結果や学習条件を記録する仕組みが必要．

- ターミナルへの出力：これで何度も失敗してきた

- 人力メモ帳ログ：めんどい 条件さぼって結局忘れる．再実験

-> ツールに頼って！

- Weight & Biases：一番トレンド\
https://wandb.ai/site/ja/

- mlflow：手軽で使いやすい\
https://mlflow.org/

- 自作：融通利くから超便利だけどコーディングめんどい

